In [1]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)
from tqdm import tqdm
import math
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score

import gc

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Statistical tests (ran after results generated)

In [2]:
set_results = [0.816038,
0.829856,
0.838451,
0.834127,
0.790073,
0.825867,
0.772416,
0.818897,
0.817904,
0.821161]
random_set_results = [0.801023,
0.809279,
0.852223,
0.828857,
0.747952,
0.773643,
0.829476,
0.776294,
0.799354,
0.824476]
lr_results = [0.789008,
0.790284,
0.819746,
0.791620,
0.760105,
0.814236,
0.776551,
0.779840,
0.777884,
0.788120]
charlson_results =[0.656873,
0.663736,
0.645588,
0.660310,
0.625806,
0.668568,
0.652461,
0.639701,
0.628784,
0.654787	
]
 

set_results_8 = [0.967391,
0.911765,
0.957143,
0.910256,
0.577381,
0.854430,
0.794118]
random_set_results_8 = [0.989130,
0.823529,
0.185714,
0.076923,
0.684524,
0.588608,
0.544118]
lr_results_8 = [0.967391,
0.963235,
0.985714,
0.974359,
0.708333,
0.568354,
0.441176]
charlson_results_8 = [0.654787,
0.492647,
0.492857,
0.500000,
0.500000,
0.493671,
0.500000]

In [3]:
from scipy.stats import shapiro 
from scipy.stats import kstest
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon
from scipy.stats import normaltest

In [39]:
shapiro(set_results)
kstest(set_results, 'norm')

ShapiroResult(statistic=0.8530228137969971, pvalue=0.06309852004051208)

KstestResult(statistic=0.7800659597749264, pvalue=6.097117198531377e-07)

In [31]:
shapiro(random_set_results)
kstest(random_set_results, 'norm')

ShapiroResult(statistic=0.9691386222839355, pvalue=0.8827458620071411)

KstestResult(statistic=0.7727554447028735, pvalue=8.696277437320612e-07)

In [32]:
shapiro(lr_results)
kstest(lr_results, 'norm')

ShapiroResult(statistic=0.9283631443977356, pvalue=0.4319867789745331)

KstestResult(statistic=0.7764040879128568, pvalue=7.29211618720243e-07)

In [4]:
shapiro(charlson_results)
kstest(charlson_results, 'norm')
normaltest(charlson_results)

ShapiroResult(statistic=0.9371631145477295, pvalue=0.5219002366065979)

KstestResult(statistic=0.7342789020541408, pvalue=4.915848430562385e-06)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


NormaltestResult(statistic=1.005837393893007, pvalue=0.6047629614707875)

In [37]:
shapiro(set_results_8)
kstest(set_results_8, 'norm')

ShapiroResult(statistic=0.8202360272407532, pvalue=0.06455416977405548)

KstestResult(statistic=0.7181589467957652, pvalue=0.0003549931229125905)

In [38]:
shapiro(random_set_results_8)
kstest(random_set_results_8, 'norm')

ShapiroResult(statistic=0.9490331411361694, pvalue=0.7208895087242126)

KstestResult(statistic=0.5306575997994031, pvalue=0.02292443342962655)

In [40]:
shapiro(lr_results_8)
kstest(lr_results_8, 'norm')

ShapiroResult(statistic=0.8019464015960693, pvalue=0.04281892254948616)

KstestResult(statistic=0.6704572067007551, pvalue=0.001245833330910244)

In [5]:
shapiro(charlson_results_8)
kstest(charlson_results_8, 'norm')
normaltest(charlson_results_8)

ShapiroResult(statistic=0.5029751062393188, pvalue=1.8005403035203926e-05)

KstestResult(statistic=0.6888689837361116, pvalue=0.0007837190773760501)

ValueError: skewtest is not valid with less than 8 samples; 7 samples were given.

In [6]:
ttest_rel(set_results, random_set_results) 
ttest_rel(set_results, lr_results) 
ttest_rel(lr_results, random_set_results) 
ttest_rel(random_set_results, lr_results) 

ttest_rel(charlson_results, set_results) 
ttest_rel(charlson_results, random_set_results) 
ttest_rel(charlson_results, lr_results) 

Ttest_relResult(statistic=1.2022451343608076, pvalue=0.2599446970742992)

Ttest_relResult(statistic=5.844200133068764, pvalue=0.00024552765660242667)

Ttest_relResult(statistic=-1.768451442569985, pvalue=0.11077672866130538)

Ttest_relResult(statistic=1.768451442569985, pvalue=0.11077672866130538)

Ttest_relResult(statistic=-25.91095431545953, pvalue=9.159242655591445e-10)

Ttest_relResult(statistic=-16.49170413084455, pvalue=4.938874103199237e-08)

Ttest_relResult(statistic=-30.09847575145, pvalue=2.412157527877996e-10)

In [7]:
wilcoxon(set_results, random_set_results) 
wilcoxon(set_results, lr_results) 
wilcoxon(lr_results, random_set_results) 

wilcoxon(charlson_results, set_results) 
wilcoxon(charlson_results, random_set_results) 
wilcoxon(charlson_results, lr_results) 

WilcoxonResult(statistic=14.0, pvalue=0.193359375)

WilcoxonResult(statistic=1.0, pvalue=0.00390625)

WilcoxonResult(statistic=13.0, pvalue=0.16015625)

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [8]:
ttest_rel(set_results_8, random_set_results_8) 
ttest_rel(set_results_8, lr_results_8) 
ttest_rel(lr_results_8, random_set_results_8) 

ttest_rel(charlson_results_8, set_results_8) 
ttest_rel(charlson_results_8, random_set_results_8) 
ttest_rel(charlson_results_8, lr_results_8) 

Ttest_relResult(statistic=2.1208077690526217, pvalue=0.07819611078960588)

Ttest_relResult(statistic=0.7313937553608119, pvalue=0.49210321625783504)

Ttest_relResult(statistic=1.5458160651212287, pvalue=0.173106274064707)

Ttest_relResult(statistic=-6.893102535820686, pvalue=0.0004603031040224992)

Ttest_relResult(statistic=-0.3288271130612035, pvalue=0.7534604483729107)

Ttest_relResult(statistic=-3.4393097652361853, pvalue=0.01381262106563483)

In [10]:
wilcoxon(set_results_8, random_set_results_8) 
wilcoxon(set_results_8, lr_results_8) 
wilcoxon(lr_results_8, random_set_results_8) 

wilcoxon(charlson_results_8, set_results_8) 
wilcoxon(charlson_results_8, random_set_results_8) 
wilcoxon(charlson_results_8, lr_results_8) 

WilcoxonResult(statistic=4.0, pvalue=0.109375)

WilcoxonResult(statistic=10.0, pvalue=0.916511907863894)

WilcoxonResult(statistic=7.0, pvalue=0.296875)

WilcoxonResult(statistic=0.0, pvalue=0.015625)

WilcoxonResult(statistic=11.0, pvalue=0.6875)

WilcoxonResult(statistic=1.0, pvalue=0.03125)

# LR

In [3]:
# Import
path = r'data/slim_problem_dummies_death.csv'
slim_problem_dummies_death = pd.read_csv(path, index_col=0)
slim_problem_dummies_death.reset_index(inplace=True, drop=True)
# Work out count for problems
problem_sum = pd.DataFrame(slim_problem_dummies_death.iloc[:,2:].sum(axis=0), columns=['Count']).sort_values(by=['Count'], ascending=False)

# Define what is a rare disease
cut_off_list = [45, 8]

In [ ]:
# Get rare 10 fold cv results for one hot

results_df = pd.DataFrame()
x_data = slim_problem_dummies_death.iloc[:,2:]
y_data = slim_problem_dummies_death.death_year_label
# Get CV folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
fold_n = 0
for train_index, test_index in cv.split(x_data, y_data):
    fold_n += 1
    x_train  = x_data[x_data.index.isin(list(train_index))]
    y_train  = y_data[y_data.index.isin(list(train_index))]
    x_test  = x_data[x_data.index.isin(list(test_index))]
    y_test  = y_data[y_data.index.isin(list(test_index))]

    # Fit
    LR = LogisticRegression(class_weight='balanced')
    LR.fit(x_train, y_train)

    # Get results for rare diseases 
    for n in cut_off_list:
        # Get filter list 
        filter_list = problem_sum[problem_sum['Count'] > n].index.tolist()
        x_test2 = x_test.copy()
        for code in filter_list:
            x_test2 = x_test2.loc[x_test2[code] != 1]
        x_test_list = x_test2.index.tolist()
        y_test2 = y_test.loc[x_test_list]

        # AUC
        try:
            aucroc = roc_auc_score(y_test2, LR.predict(x_test2))
        except:
            aucroc = np.nan
        # Accuracy
        accuracy = accuracy_score(y_test2, LR.predict(x_test2))
        # Precision
        precision = precision_score(y_test2, LR.predict(x_test2))
        # Recall
        recall = precision_score(y_test2, LR.predict(x_test2))
        # AUPRC
        try:
            auprc = average_precision_score(y_test2, LR.predict(x_test2))
        except:
            auprc = np.nan
        # F1
        f1 = f1_score(y_test2, LR.predict(x_test2))
        # TPR and FPR
        cm = confusion_matrix(y_test2, LR.predict(x_test2))
        try:
            _tp = cm[0, 0]
        except:
            _tp = np.nan
        try:
            _fn = cm[0, 1]
        except:
            _fn = np.nan
        try:
            _fp = cm[1, 0]
        except:
            _fp = np.nan
        try:
            _tn = cm[1, 1]
        except:
            _tn = np.nan
        tpr = _tp / (_tp + _fn)
        fpr = _fp / (_tn + _fp)

        new_row = {'data': 'dummies', 'fold': fold_n, 'subset': n, 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
        results_df = results_df.append(new_row, ignore_index=True)

In [8]:
results_df

,data,fold,subset,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
0,dummies,1,45,0.868043,0.901996,0.086207,0.086207,0.156250,0.073654,0.902752,0.166667
1,dummies,1,8,NaN,0.946237,0.000000,0.000000,0.000000,-0.000000,0.946237,NaN
2,dummies,2,45,0.939479,0.880295,0.084507,0.084507,0.155844,0.084507,0.878957,0.000000
3,dummies,2,8,NaN,0.920000,0.000000,0.000000,0.000000,-0.000000,0.920000,NaN
4,dummies,3,45,0.769104,0.867961,0.084507,0.084507,0.150000,0.062163,0.871542,0.333333
5,dummies,3,8,0.967391,0.935484,0.142857,0.142857,0.250000,0.142857,0.934783,0.000000
6,dummies,4,45,0.843648,0.885542,0.126984,0.126984,0.219178,0.105603,0.887295,0.200000
7,dummies,4,8,0.963235,0.929577,0.375000,0.375000,0.545455,0.375000,0.926471,0.000000
8,dummies,5,45,0.785374,0.899800,0.113208,0.113208,0.193548,0.081484,0.904082,0.333333
9,dummies,5,8,0.985714,0.971831,0.333333,0.333333,0.500000,0.333333,0.971429,0.000000


In [9]:
results_df.groupby(['data', 'subset']).mean()
results_df.groupby(['data', 'subset']).std()

fold     AUROC  accuracy  precision    recall        F1     AUPRC       TPR       FPR
data    subset                                                                                       
dummies 8        5.5  0.801223  0.917960   0.134286  0.134286  0.201061  0.123484  0.925349  0.328571
        45       5.5  0.785632  0.879998   0.092882  0.092882  0.162536  0.070018  0.883976  0.312713

fold     AUROC  accuracy  precision    recall        F1     AUPRC       TPR       FPR
data    subset                                                                                          
dummies 8       3.02765  0.227436  0.036319   0.139531  0.139531  0.208150  0.138454  0.030417  0.434796
        45      3.02765  0.079854  0.014437   0.016060  0.016060  0.025554  0.016671  0.013523  0.157587

In [ ]:
# Get 10 fold cv results for one hot
# Import
path = r'data/slim_problem_dummies_death.csv'
slim_problem_dummies_death = pd.read_csv(path, index_col=0)
slim_problem_dummies_death.reset_index(inplace=True, drop=True)

results_df = pd.DataFrame()
x_data = slim_problem_dummies_death.iloc[:,2:]
y_data = slim_problem_dummies_death.death_year_label
# Get CV folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
fold_n = 0
for train_index, test_index in cv.split(x_data, y_data):
    fold_n += 1
    x_train  = x_data[x_data.index.isin(list(train_index))]
    y_train  = y_data[y_data.index.isin(list(train_index))]
    x_test  = x_data[x_data.index.isin(list(test_index))]
    y_test  = y_data[y_data.index.isin(list(test_index))]

    # Fit
    LR = LogisticRegression(class_weight='balanced')
    LR.fit(x_train, y_train)

    # AUC
    aucroc = roc_auc_score(y_test, LR.predict(x_test))
    # Accuracy
    accuracy = accuracy_score(y_test, LR.predict(x_test))
    # Precision
    precision = precision_score(y_test, LR.predict(x_test))
    # Recall
    recall = recall_score(y_test, LR.predict(x_test))
    # AUPRC
    auprc = average_precision_score(y_test, LR.predict(x_test))
    # F1
    f1 = f1_score(y_test, LR.predict(x_test))
    # TPR and FPR
    #fpr, tpr, thresholds = roc_curve(y_test, LR.predict(x_test))
    cm = confusion_matrix(y_test, LR.predict(x_test))
    _tp = cm[0, 0]
    _fn = cm[0, 1]
    _fp = cm[1, 0]
    _tn = cm[1, 1]
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'dummies', 'fold': fold_n, 'subset':'overall', 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)

In [5]:
results_df

,data,fold,subset,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
0,dummies,1,overall,0.789008,0.849878,0.113621,0.724891,0.196450,0.089327,0.853125,0.275109
1,dummies,2,overall,0.790284,0.848220,0.112991,0.729258,0.195665,0.089253,0.851310,0.270742
2,dummies,3,overall,0.819746,0.851758,0.122283,0.786026,0.211640,0.101534,0.853465,0.213974
3,dummies,4,overall,0.791620,0.857948,0.119683,0.721739,0.205318,0.093455,0.861502,0.278261
4,dummies,5,overall,0.760105,0.847430,0.105010,0.668122,0.181495,0.078562,0.852087,0.331878
5,dummies,6,overall,0.814236,0.857601,0.124734,0.768559,0.214634,0.101725,0.859914,0.231441
6,dummies,7,overall,0.776551,0.846324,0.108564,0.703057,0.188084,0.083844,0.850045,0.296943
7,dummies,8,overall,0.779840,0.844444,0.108378,0.711790,0.188113,0.084439,0.847890,0.288210
8,dummies,9,overall,0.777884,0.853068,0.112676,0.698690,0.194057,0.086354,0.857078,0.301310
9,dummies,10,overall,0.788120,0.852294,0.114823,0.720524,0.198079,0.089808,0.855717,0.279476


In [6]:
results_df.groupby(['data', 'subset']).mean()
results_df.groupby(['data', 'subset']).std()

,,fold,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
data,subset,,,,,,,,,
dummies,overall,5.5,0.788739,0.850897,0.114276,0.723266,0.197354,0.08983,0.854213,0.276734


,,fold,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
data,subset,,,,,,,,,
dummies,overall,3.02765,0.017601,0.004532,0.006333,0.033704,0.010562,0.007425,0.004322,0.033704


# Set transformer

In [9]:
# Import
path = r'data/slim_problem_dummies_death.csv'
slim_problem_dummies_death = pd.read_csv(path, index_col=0)
# Work out count for problems
problem_sum = pd.DataFrame(slim_problem_dummies_death.iloc[:,2:].sum(axis=0), columns=['Count']).sort_values(by=['Count'], ascending=False)
# Define what is a rare disease
cut_off_list = [45, 8]
# Get indexes for rare diseases 
filter_list_45 = problem_sum[problem_sum['Count'] > 45].index.tolist()
filter_list_8 = problem_sum[problem_sum['Count'] > 8].index.tolist()

problem_dummies_45 = slim_problem_dummies_death.iloc[:,2:].reset_index().copy()
del slim_problem_dummies_death
print('Working on 45...')
for code in filter_list_45:
    problem_dummies_45 = problem_dummies_45.loc[problem_dummies_45[code] != 1]
    index_list_45 = problem_dummies_45.index.tolist()
del problem_dummies_45

path = r'data/slim_problem_dummies_death.csv'
slim_problem_dummies_death = pd.read_csv(path, index_col=0)
problem_dummies_8 = slim_problem_dummies_death.iloc[:,2:].reset_index().copy()
del slim_problem_dummies_death
print('Working on 8...')
for code in filter_list_8:
    problem_dummies_8 = problem_dummies_8.loc[problem_dummies_8[code] != 1]
    index_list_8 = problem_dummies_8.index.tolist()
del problem_dummies_8

Working on 45...
Working on 8...


In [10]:
# Set transformer initial setup

# Import
path = r'data/slim_problem_dummies_death.csv'
slim_problem_dummies_death = pd.read_csv(path, index_col=0)
# Import
path = r'data/final_trimmed_snomed_embedding_128d.csv'
snomed_embedding = pd.read_csv(path)

random_bool = True
#random_bool = False
if random_bool == True:
    # Get random embeddings for each disease
    snomed_embedding.set_index(['snomed_code'], inplace=True)
    np.random.seed(0)
    random_embedding = pd.DataFrame(np.random.default_rng(seed=0).uniform(low=snomed_embedding.min().min(),high=snomed_embedding.max().max(),size=[len(snomed_embedding),len(snomed_embedding.columns)]))
    random_embedding.index = snomed_embedding.index
    random_embedding.columns = snomed_embedding.columns
    random_embedding.reset_index(inplace=True)
    snomed_embedding = random_embedding

# Create dfs
patient_mortality = slim_problem_dummies_death.iloc[:,:2]
patient_df = slim_problem_dummies_death.iloc[:,2:]
# Str
patient_df.columns = patient_df.columns.astype(str)
snomed_embedding['snomed_code'] = snomed_embedding['snomed_code'].astype(str)
# Filter
snomed_embedding = snomed_embedding[snomed_embedding['snomed_code'].isin(patient_df.columns.tolist())]
snomed_embedding.set_index('snomed_code', inplace=True)

# Get lengths of each patients co-morbidities
comorbidity_len = np.array(patient_df.sum(axis=1))

# Add padding embedding 
padding_df = pd.DataFrame(np.random.choice([0], size=len(snomed_embedding.columns)))
padding_df = padding_df.T
padding_df.index = ['9999999999']
padding_df.columns = snomed_embedding.columns
snomed_embedding2 = pd.concat([snomed_embedding, padding_df])
snomed_embedding2.index = snomed_embedding2.index.astype(str)

# Get max number of co-morbidities
max_len = int(patient_df.sum(axis=1).max())

# Format patients embeddings into set and pad / create array
feature_array = np.zeros(shape=(len(patient_df), max_len , 128))
n = -1
for index, row in patient_df.iterrows():
    n += 1
    n2 = -1
    code_list = row[row ==1].index.tolist()
    while len(code_list) < max_len:
        code_list.append('9999999999')
    for code in code_list:
        n2 += 1
        feature_array[n, n2] = np.array(snomed_embedding2.loc[code])

# Get array for death
mortality_array = np.array(patient_mortality['death_year_label'])
mortality_array = mortality_array.squeeze()

# Create mask tensor based on lengths
comorbidity_len2 = torch.as_tensor(comorbidity_len, dtype=torch.long)
mask = torch.arange(max_len)[None, :] < comorbidity_len2[:, None]

# del so more memory
del slim_problem_dummies_death
del snomed_embedding
del patient_mortality
del snomed_embedding2
del padding_df
del row
del comorbidity_len
del comorbidity_len2
gc.collect()

57187

In [3]:
# Check if any have no co-morbidities
for n in range(mask.shape[0]):
    if torch.all(mask[n] == False):
        print(n)

In [11]:
# Custom dataset class
class DiseaseDataset(Dataset):
    def __init__(self, disease_embeddings, mortality_labels, padding_mask):
        self.disease_embeddings = disease_embeddings
        self.mortality_labels = mortality_labels
        self.padding_mask = padding_mask

    def __len__(self):
        return len(self.mortality_labels)

    def __getitem__(self, idx):
        return self.disease_embeddings[idx], self.mortality_labels[idx], self.padding_mask[idx] 

In [12]:
class SetTransformer(nn.Module):
    def __init__(self, dim_input, num_outputs, dim_output,
            num_inds=36, dim_hidden=160, num_heads=4, ln=False):
        super(SetTransformer, self).__init__()
        self.enc = nn.Sequential(
                ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln),
                ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln))
        self.isab = ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)
        self.pma = PMA(dim_hidden, num_heads, num_outputs, ln=ln)
        self.dec = nn.Sequential(
                #SAB(dim_hidden, dim_hidden, num_heads, ln=ln),
                #SAB(dim_hidden, dim_hidden, num_heads, ln=ln),
                nn.Linear(dim_hidden, dim_output))

    def forward(self, X, batch_mask):
        x = self.isab(X, batch_mask)
        x = self.pma(x, batch_mask)
        return self.dec(x), x

class MAB0(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False):
        super(MAB0, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Linear(dim_V, dim_V)

    def forward(self, Q, K, mask):
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)

        dim_split = self.dim_V // self.num_heads
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)

        # Create new variable for softmax
        WB_ = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)

        # Exspand mask dimensions to align
        mask = mask.unsqueeze(1).repeat(self.num_heads, Q.shape[1], 1)
        # Mask for softmax
        WB_[~mask] = float('-inf')

        A = torch.softmax(WB_, 2)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + F.relu(self.fc_o(O))
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Linear(dim_V, dim_V)

    def forward(self, Q, K):
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)

        dim_split = self.dim_V // self.num_heads
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)

        A = torch.softmax(Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V), 2)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + F.relu(self.fc_o(O))
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False):
        super(SAB, self).__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln)

    def forward(self, X):
        return self.mab(X, X)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)
        self.mab0 = MAB0(dim_out, dim_in, dim_out, num_heads, ln=ln)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln)

    def forward(self, X, mask):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, mask)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB0(dim, dim, dim, num_heads, ln=ln)

    def forward(self, X, mask):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, mask)

In [13]:
# Define how long an epoch takes
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Train function 
def train(model, dataloader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    batch_prediction_list = []
    batch_label_list = []
    
    for i, (batch_embeddings, batch_labels, batch_mask) in enumerate(tqdm(dataloader, 0)):
        batch_labels = batch_labels.to(device)
        batch_embeddings = batch_embeddings.to(device)
        batch_mask = batch_mask.to(device)

        optimizer.zero_grad()
        logits, pma = model(batch_embeddings, batch_mask)
        logits = logits.squeeze(-1) # squeeze to remove extra dimensions
        batch_labels = batch_labels.unsqueeze(1)
        
        loss = criterion(logits, batch_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        sig = torch.nn.Sigmoid()
        output = sig(logits)      
        np_predictions = output.cpu().detach().numpy()
        np_labels = batch_labels.cpu().detach().numpy()
        np_pma = pma.cpu().detach().numpy()

        np_predictions = np_predictions.squeeze()
        np_labels = np_labels.squeeze()
        np_pma = np_pma.squeeze()

        np_predictions = np_predictions.flatten()
        np_labels = np_labels.flatten()
        
        # Create list
        for x in np_predictions:
            batch_prediction_list.append(x)
        for x in np_labels:
            batch_label_list.append(x)
        if i == 0:
            np_pma_final = np_pma
        else:
            np_pma_final = np.vstack([np_pma_final, np_pma])

    final_predictions = np.array(batch_prediction_list)

    final_labels = np.array(batch_label_list)

    try:
        auroc = roc_auc_score(final_labels, final_predictions)
    except:
        auroc = np.nan
    
    try:
        final_loss = epoch_loss / len(dataloader)
    except:
        final_loss = np.nan

    return final_loss, auroc, final_predictions, final_labels, np_pma_final

# Eval function
def evaluate(model, dataloader, criterion):

    # Set the model to evaluation mode
    model.eval()
    epoch_loss = 0
    batch_prediction_list = []
    batch_label_list = []

    # use the with torch.no_grad() block to ensure no gradients are calculated within the bloc
    with torch.no_grad():
        for i, (batch_embeddings, batch_labels, batch_mask) in enumerate(tqdm(dataloader, 0)):
            batch_labels = batch_labels.to(device)
            batch_embeddings = batch_embeddings.to(device)
            batch_mask = batch_mask.to(device)

            logits, pma = model(batch_embeddings, batch_mask)
            logits = logits.squeeze() # squeeze to remove extra dimensions

            if len(logits.size()) == 0: # Need to add this so tensor with just one element does not cause error
                logits = logits.unsqueeze(dim=0)

            loss = criterion(logits, batch_labels)

            epoch_loss += loss.item()

            sig = torch.nn.Sigmoid()
            output = sig(logits)      
            np_predictions = output.cpu().detach().numpy()
            np_labels = batch_labels.cpu().detach().numpy()
            np_pma = pma.cpu().detach().numpy()

            np_predictions = np_predictions.squeeze()
            np_labels = np_labels.squeeze()

            np_predictions = np_predictions.flatten()
            np_labels = np_labels.flatten()
            np_pma = np_pma.squeeze()
            
            # Create list
            for x in np_predictions:
                batch_prediction_list.append(x)
            for x in np_labels:
                batch_label_list.append(x)
            if i == 0:
                np_pma_final = np_pma
            else:
                np_pma_final = np.vstack([np_pma_final, np_pma])

        final_predictions = np.array(batch_prediction_list)

        final_labels = np.array(batch_label_list)

        try:
            auroc = roc_auc_score(final_labels, final_predictions)
        except:
            auroc = np.nan
        
        try:
            final_loss = epoch_loss / len(dataloader)
        except:
            final_loss = np.nan

    return final_loss, auroc, final_predictions, final_labels, np_pma_final

In [6]:
# CV for set transformer

# Get CV folds and create dictionarys 
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
fold_n = 0
train_index_dict = {}
test_index_dict = {}
for train_index, test_index in cv.split(feature_array, mortality_array):
    fold_n += 1
    train_index_dict[fold_n] = train_index
    test_index_dict[fold_n] = test_index

In [7]:
# del for memmory 
del test_index
del train_index
del patient_df
del index
del fold_n
del code
del code_list
gc.collect()

190

In [8]:
# Main run
best_test_auroc = 0
results_df = pd.DataFrame()
for fold in range(10):
    fold += 1
    if fold < 10:
        # Get train, validation and test sets
        train_embeddings = feature_array[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_embeddings = feature_array[test_index_dict[fold+1]]
        test_embeddings = feature_array[test_index_dict[fold]]

        train_labels = mortality_array[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_labels = mortality_array[test_index_dict[fold+1]]
        test_labels = mortality_array[test_index_dict[fold]]

        # Split masks
        train_padding_mask = mask[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_padding_mask = mask[test_index_dict[fold+1]]
        test_padding_mask = mask[test_index_dict[fold]]
    else: # Change here to fist cv split for final fold
        # Get train, validation and test sets
        train_embeddings = feature_array[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_embeddings = feature_array[test_index_dict[1]]
        test_embeddings = feature_array[test_index_dict[fold]]

        train_labels = mortality_array[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_labels = mortality_array[test_index_dict[1]]
        test_labels = mortality_array[test_index_dict[fold]]

        # Split masks
        train_padding_mask = mask[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_padding_mask = mask[test_index_dict[1]]
        test_padding_mask = mask[test_index_dict[fold]]
    
    # Create datasets
    train_dataset = DiseaseDataset(train_embeddings.astype(np.float32), train_labels.astype(np.float32), train_padding_mask)
    val_dataset = DiseaseDataset(val_embeddings.astype(np.float32), val_labels.astype(np.float32), val_padding_mask)
    test_dataset = DiseaseDataset(test_embeddings.astype(np.float32), test_labels.astype(np.float32), test_padding_mask)

    # Define batch size
    batch_size = 512

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Create weight for loss
    unique, counts = np.unique(train_labels, return_counts=True)
    pos_weight = torch.Tensor([(counts[0] / counts[1])])

    # Define model
    model = SetTransformer(dim_input=128, num_outputs=1, dim_output=1, num_inds=32, dim_hidden=160, num_heads=4, ln=False)

    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Run
    best_valid_loss = float('inf')
    best_valid_auroc = 0
    num_epochs = 10

    for epoch in range(num_epochs):

        start_time = time.time()

        train_loss, train_auroc, train_predictions, train_labels_out, train_pma_final = train(model, train_loader, optimizer, criterion)
        valid_loss, valid_auroc, valid_predictions, valid_labels_out, valid_pma_final = evaluate(model, val_loader, criterion)
        
        if epoch % 1 == 0:
            print('Epoch:', epoch)
            print(f'train loss: {train_loss:.3f}')
            print(f'train AUROC: {train_auroc:.3f}')
            print(f'valid loss: {valid_loss:.3f}')
            print(f'valid AUROC: {valid_auroc:.3f}')
        
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss  
            print('BEST VALID LOSS:', best_valid_loss)

        if valid_auroc > best_valid_auroc:
            best_valid_auroc = valid_auroc
            print('BEST VALID AUROC:', best_valid_auroc)
            print('UPDATED BEST INTERMEDIATE MODEL')
            torch.save(model.state_dict(), f'intermediate_set_transformer_mortality.pt')

    # -----------------------------
    # Evaluate best model on test set
    # -----------------------------

    model.load_state_dict(torch.load(f'intermediate_set_transformer_mortality.pt'))

    test_loss, test_auroc, test_predictions, test_labels_out, test_pma_final = evaluate(model, test_loader, criterion)

    # del for memmory 
    del train_embeddings
    del val_embeddings
    del test_embeddings
    del train_labels
    del val_labels
    del test_labels
    del train_padding_mask
    del val_padding_mask
    del test_padding_mask
    del train_loader
    del val_loader
    del test_loader
    del train_predictions
    del train_labels_out
    del train_pma_final
    del valid_predictions
    del valid_labels_out
    del valid_pma_final
    gc.collect()

    print(f'test predictions: {test_predictions.mean().item():.3f}') 
    print(f'test loss: {test_loss:.3f}')
    print(f'test AUROC: {test_auroc:.3f}')

    if test_auroc > best_test_auroc:
        best_test_auroc = test_auroc
        print('BEST TEST AUROC:', best_test_auroc)
        print('UPDATED BEST MODEL')
        torch.save(model.state_dict(), f'set_transformer_mortality.pt')

    # Get results
    # AUC
    aucroc = roc_auc_score(test_labels_out, test_predictions)
    # Accuracy
    accuracy = accuracy_score(test_labels_out, test_predictions.round())
    # Precision
    precision = precision_score(test_labels_out, test_predictions.round())
    # Recall
    recall = recall_score(test_labels_out, test_predictions.round())
    # AUPRC
    auprc = average_precision_score(test_labels_out, test_predictions)
    # F1
    f1 = f1_score(test_labels_out, test_predictions.round())
    # TPR and FPR
    cm = confusion_matrix(test_labels_out, test_predictions.round())
    _tp = cm[0, 0]
    _fn = cm[0, 1]
    _fp = cm[1, 0]
    _tn = cm[1, 1]
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'set', 'fold': fold, 'subset':'overall', 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)


100%|██████████| 18/18 [00:02<00:00,  6.07it/s]


Epoch: 0
train loss: 1.142
train AUROC: 0.771
valid loss: 1.247
valid AUROC: 0.781
BEST VALID LOSS: 1.2471927205721538
BEST VALID AUROC: 0.7809249004379689
UPDATED BEST INTERMEDIATE MODEL
Epoch: 1
train loss: 1.035
train AUROC: 0.813
valid loss: 1.003
valid AUROC: 0.840
BEST VALID LOSS: 1.003496958149804
BEST VALID AUROC: 0.8396168477628321
UPDATED BEST INTERMEDIATE MODEL
Epoch: 2
train loss: 0.962
train AUROC: 0.838
valid loss: 1.251
valid AUROC: 0.801
Epoch: 3
train loss: 1.148
train AUROC: 0.748
valid loss: 1.020
valid AUROC: 0.826
Epoch: 4
train loss: 0.977
train AUROC: 0.827
valid loss: 0.948
valid AUROC: 0.843
BEST VALID LOSS: 0.9478144513236152
BEST VALID AUROC: 0.8434519856192856
UPDATED BEST INTERMEDIATE MODEL
Epoch: 5
train loss: 0.944
train AUROC: 0.841
valid loss: 1.130
valid AUROC: 0.812
Epoch: 6
train loss: 1.002
train AUROC: 0.823
valid loss: 0.957
valid AUROC: 0.847
BEST VALID AUROC: 0.8472447777294061
UPDATED BEST INTERMEDIATE MODEL
Epoch: 7
train loss: 0.918
train AUR

<All keys matched successfully>

100%|██████████| 18/18 [00:02<00:00,  6.14it/s]


0

...

<All keys matched successfully>

100%|██████████| 18/18 [00:03<00:00,  5.98it/s]


0

test predictions: 0.366
test loss: 1.184
test AUROC: 0.821


/tmp/ipykernel_6357/285864036.py:161: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)


In [9]:
results_df

,data,fold,subset,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
0,set,1,overall,0.816038,0.778466,0.075562,0.689956,0.136207,0.083026,0.780764,0.310044
1,set,2,overall,0.829856,0.806765,0.082921,0.659389,0.147317,0.120749,0.810593,0.340611
2,set,3,overall,0.838451,0.765974,0.075540,0.733624,0.136975,0.135031,0.766814,0.266376
3,set,4,overall,0.834127,0.906367,0.130539,0.473913,0.204695,0.135992,0.917650,0.526087
4,set,5,overall,0.790073,0.482255,0.044581,0.951965,0.085173,0.082219,0.470054,0.048035
5,set,6,overall,0.825867,0.905583,0.131920,0.489083,0.207792,0.136805,0.916402,0.510917
6,set,7,overall,0.772416,0.307131,0.034392,0.973799,0.066438,0.098275,0.289814,0.026201
7,set,8,overall,0.818897,0.505473,0.045610,0.930131,0.086957,0.092536,0.494442,0.069869
8,set,9,overall,0.817904,0.907573,0.112388,0.384279,0.173913,0.101250,0.921166,0.615721
9,set,10,overall,0.821161,0.649530,0.058010,0.842795,0.108549,0.113101,0.644510,0.157205


In [11]:
# Save
#results_df.to_csv('set_transformer_mortality_10_fold_cv_results')

In [13]:
results_df.groupby(['data', 'subset']).mean()
results_df.groupby(['data', 'subset']).std()

,,fold,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
data,subset,,,,,,,,,
set,overall,5.5,0.816479,0.701512,0.079146,0.712893,0.135402,0.109898,0.701221,0.287107


,,fold,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
data,subset,,,,,,,,,
set,overall,3.02765,0.020358,0.208692,0.035497,0.212995,0.04946,0.02151,0.219287,0.212995


In [9]:
# CV for set transformer - only testing on those with rare co-morbidities

# Get CV folds and create dictionarys 
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
fold_n = 0
train_index_dict = {}
test_index_dict = {}
test_index_dict_8 = {}
test_index_dict_45 = {}

for train_index, test_index in cv.split(feature_array, mortality_array):
    fold_n += 1
    train_index_dict[fold_n] = train_index
    test_index_dict[fold_n] = test_index
    # Filter for rare diseases in test set
    test_index_8 = np.array(list(set(test_index.tolist()) & set(index_list_8)))
    test_index_45 = np.array(list(set(test_index.tolist()) & set(index_list_45)))
    test_index_dict_8[fold_n] = test_index_8
    test_index_dict_45[fold_n] = test_index_45

In [10]:
# del for memmory 
del test_index
del train_index
del patient_df
del index
del fold_n
del code
del code_list
del path
del n 
del n2
del max_len
gc.collect()

471

In [12]:
# Run
best_test_auroc = 0
results_df = pd.DataFrame()
for fold in range(10):
    fold += 1
    if fold < 10:
        # Get train, validation and test sets
        train_embeddings = feature_array[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_embeddings = feature_array[test_index_dict[fold+1]]
        test_embeddings_8 = feature_array[test_index_dict_8[fold]]
        test_embeddings_45 = feature_array[test_index_dict_45[fold]]

        train_labels = mortality_array[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_labels = mortality_array[test_index_dict[fold+1]]
        test_labels_8 = mortality_array[test_index_dict_8[fold]]
        test_labels_45 = mortality_array[test_index_dict_45[fold]]    

        # Split masks
        train_padding_mask = mask[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_padding_mask = mask[test_index_dict[fold+1]]
        test_padding_mask_8 = mask[test_index_dict_8[fold]]
        test_padding_mask_45 = mask[test_index_dict_45[fold]]

    else: # Change here to fist cv split for final fold
        # Get train, validation and test sets
        train_embeddings = feature_array[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_embeddings = feature_array[test_index_dict[1]]
        test_embeddings_8 = feature_array[test_index_dict_8[fold]]
        test_embeddings_45 = feature_array[test_index_dict_45[fold]]

        train_labels = mortality_array[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_labels = mortality_array[test_index_dict[1]]
        test_labels_8 = mortality_array[test_index_dict_8[fold]]
        test_labels_45 = mortality_array[test_index_dict_45[fold]]

        # Split masks
        train_padding_mask = mask[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_padding_mask = mask[test_index_dict[1]]
        test_padding_mask_8 = mask[test_index_dict_8[fold]]
        test_padding_mask_45 = mask[test_index_dict_45[fold]]
    
    # Create datasets
    train_dataset = DiseaseDataset(train_embeddings.astype(np.float32), train_labels.astype(np.float32), train_padding_mask)
    val_dataset = DiseaseDataset(val_embeddings.astype(np.float32), val_labels.astype(np.float32), val_padding_mask)
    test_dataset_8 = DiseaseDataset(test_embeddings_8.astype(np.float32), test_labels_8.astype(np.float32), test_padding_mask_8)
    test_dataset_45 = DiseaseDataset(test_embeddings_45.astype(np.float32), test_labels_45.astype(np.float32), test_padding_mask_45)

    # Define batch size
    batch_size = 512

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader_8 = DataLoader(test_dataset_8, batch_size=batch_size)
    test_loader_45 = DataLoader(test_dataset_45, batch_size=batch_size)

    # Create weight for loss
    unique, counts = np.unique(train_labels, return_counts=True)
    pos_weight = torch.Tensor([(counts[0] / counts[1])])

    # Define model
    model = SetTransformer(dim_input=128, num_outputs=1, dim_output=1, num_inds=32, dim_hidden=160, num_heads=4, ln=False)

    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Run
    best_valid_loss = float('inf')
    best_valid_auroc = 0
    num_epochs = 10

    for epoch in range(num_epochs):

        start_time = time.time()

        train_loss, train_auroc, train_predictions, train_labels_out, train_pma_final = train(model, train_loader, optimizer, criterion)
        valid_loss, valid_auroc, valid_predictions, valid_labels_out, valid_pma_final = evaluate(model, val_loader, criterion)
        
        if epoch % 1 == 0:
            print('Epoch:', epoch)
            print(f'train loss: {train_loss:.3f}')
            print(f'train AUROC: {train_auroc:.3f}')
            print(f'valid loss: {valid_loss:.3f}')
            print(f'valid AUROC: {valid_auroc:.3f}')
        
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss  
            print('BEST VALID LOSS:', best_valid_loss)

        if valid_auroc > best_valid_auroc:
            best_valid_auroc = valid_auroc
            print('BEST VALID AUROC:', best_valid_auroc)
            print('UPDATED BEST INTERMEDIATE MODEL')
            torch.save(model.state_dict(), f'intermediate_set_transformer_mortality.pt')

    # -----------------------------
    # Evaluate best model on test set
    # -----------------------------

    model.load_state_dict(torch.load(f'intermediate_set_transformer_mortality.pt'))

    test_loss_8, test_auroc_8, test_predictions_8, test_labels_out_8, test_pma_final_8 = evaluate(model, test_loader_8, criterion)
    test_loss_45, test_auroc_45, test_predictions_45, test_labels_out_45, test_pma_final_45 = evaluate(model, test_loader_45, criterion)

    # del for memmory 
    del train_embeddings
    del val_embeddings
    del train_labels
    del val_labels
    del train_padding_mask
    del val_padding_mask
    del train_loader
    del val_loader
    del train_predictions
    del train_labels_out
    del train_pma_final
    del valid_predictions
    del valid_labels_out
    del valid_pma_final
    gc.collect()

    print(f'test predictions 8: {test_predictions_8.mean().item():.3f}') 
    print(f'test loss 8: {test_loss_8:.3f}')
    print(f'test AUROC 8: {test_auroc_8:.3f}')

    print(f'test predictions 45: {test_predictions_45.mean().item():.3f}') 
    print(f'test loss 45: {test_loss_45:.3f}')
    print(f'test AUROC 45: {test_auroc_45:.3f}')

    #if test_auroc > best_test_auroc:
    #    best_test_auroc = test_auroc
    #    print('BEST TEST AUROC:', best_test_auroc)
        #print('UPDATED BEST MODEL')
        #torch.save(model.state_dict(), f'set_transformer_mortality.pt')

    # Get results
    # AUC
    try:
        aucroc = roc_auc_score(test_labels_out_8, test_predictions_8)
    except:
        aucroc = np.nan
    # Accuracy
    accuracy = accuracy_score(test_labels_out_8, test_predictions_8.round())
    # Precision
    precision = precision_score(test_labels_out_8, test_predictions_8.round())
    # Recall
    recall = recall_score(test_labels_out_8, test_predictions_8.round())
    # AUPRC
    try:
        auprc = average_precision_score(test_labels_out_8, test_predictions_8)
    except:
        auprc = np.nan
    # F1
    f1 = f1_score(test_labels_out_8, test_predictions_8.round())
    # TPR and FPR
    cm = confusion_matrix(test_labels_out_8, test_predictions_8.round())
    try:
        _tp = cm[0, 0]
    except:
        _tp = np.nan
    try:
        _fn = cm[0, 1]
    except:
        _fn = np.nan
    try:
        _fp = cm[1, 0]
    except:
        _fp = np.nan
    try:
        _tn = cm[1, 1]
    except:
        _tn = np.nan
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'set', 'fold': fold, 'subset': 8, 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)

    # Get results
    # AUC
    try:
        aucroc = roc_auc_score(test_labels_out_45, test_predictions_45)
    except:
        aucroc = np.nan
    # Accuracy
    accuracy = accuracy_score(test_labels_out_45, test_predictions_45.round())
    # Precision
    precision = precision_score(test_labels_out_45, test_predictions_45.round())
    # Recall
    recall = recall_score(test_labels_out_45, test_predictions_45.round())
    # AUPRC
    try:
        auprc = average_precision_score(test_labels_out_45, test_predictions_45)
    except:
        auprc = np.nan
    # F1
    f1 = f1_score(test_labels_out_45, test_predictions_45.round())
    # TPR and FPR
    cm = confusion_matrix(test_labels_out_45, test_predictions_45.round())
    try:
        _tp = cm[0, 0]
    except:
        _tp = np.nan
    try:
        _fn = cm[0, 1]
    except:
        _fn = np.nan
    try:
        _fp = cm[1, 0]
    except:
        _fp = np.nan
    try:
        _tn = cm[1, 1]
    except:
        _tn = np.nan
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'set', 'fold': fold, 'subset': 45, 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)


100%|██████████| 18/18 [00:03<00:00,  5.85it/s]


Epoch: 0
train loss: 1.167
train AUROC: 0.757
valid loss: 1.356
valid AUROC: 0.771
BEST VALID LOSS: 1.3558742437097762
BEST VALID AUROC: 0.7708357168292892
UPDATED BEST INTERMEDIATE MODEL
Epoch: 1
train loss: 1.066
train AUROC: 0.806
valid loss: 1.003
valid AUROC: 0.831
BEST VALID LOSS: 1.0033313499556646
BEST VALID AUROC: 0.8308270099495169
UPDATED BEST INTERMEDIATE MODEL
Epoch: 2
train loss: 0.962
train AUROC: 0.837
valid loss: 1.026
valid AUROC: 0.832
BEST VALID AUROC: 0.832040426072499
UPDATED BEST INTERMEDIATE MODEL
Epoch: 3
train loss: 0.973
train AUROC: 0.834
valid loss: 0.954
valid AUROC: 0.841
BEST VALID LOSS: 0.9538111024432712
BEST VALID AUROC: 0.8407101604532332
UPDATED BEST INTERMEDIATE MODEL
Epoch: 4
train loss: 0.985
train AUROC: 0.826
valid loss: 0.976
valid AUROC: 0.851
BEST VALID AUROC: 0.850820640753051
UPDATED BEST INTERMEDIATE MODEL
Epoch: 5
train loss: 0.950
train AUROC: 0.843
valid loss: 1.028
valid AUROC: 0.844
Epoch: 6
train loss: 0.959
train AUROC: 0.839
valid

<All keys matched successfully>

100%|██████████| 2/2 [00:00<00:00, 10.60it/s]


0

...

<All keys matched successfully>

100%|██████████| 1/1 [00:00<00:00,  5.76it/s]


0

test predictions 8: 0.290
test loss 8: 0.464
test AUROC 8: nan
test predictions 45: 0.285
test loss 45: 1.092
test AUROC 45: 0.734


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/tmp/ipykernel_23773/867624934.py:190: RuntimeWarning: invalid value encountered in long_scalars
  fpr = _fp / (_tn + _fp)
/tmp/ipykernel_23773/867624934.py:193: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
/tmp/ipykernel_23773/867624934.py:236: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [13]:
results_df

,data,fold,subset,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
0,set,1,8,NaN,0.752688,0.000000,0.000000,0.000000,-0.000000,0.752688,NaN
1,set,1,45,0.735168,0.791289,0.017699,0.333333,0.033613,0.110905,0.796330,0.666667
2,set,2,8,NaN,0.706667,0.000000,0.000000,0.000000,-0.000000,0.706667,NaN
3,set,2,45,0.868870,0.769797,0.038760,0.833333,0.074074,0.156789,0.769088,0.166667
4,set,3,8,0.967391,0.870968,0.076923,1.000000,0.142857,0.250000,0.869565,0.000000
5,set,3,45,0.922596,0.844660,0.082353,0.777778,0.148936,0.214279,0.845850,0.222222
6,set,4,8,0.911765,0.647887,0.107143,1.000000,0.193548,0.268687,0.632353,0.000000
7,set,4,45,0.893443,0.724900,0.055944,0.800000,0.104575,0.226468,0.723361,0.200000
8,set,5,8,0.957143,0.647887,0.038462,1.000000,0.074074,0.250000,0.642857,0.000000
9,set,5,45,0.778345,0.673347,0.041667,0.777778,0.079096,0.055274,0.671429,0.222222


In [14]:
results_df.groupby(['data', 'subset']).mean()
results_df.groupby(['data', 'subset']).std()

fold     AUROC  accuracy  precision    recall        F1     AUPRC       TPR       FPR
data subset                                                                                       
set  8        5.5  0.853212  0.745680   0.054376  0.590000  0.096823  0.153954  0.745516  0.157143
     45       5.5  0.795891  0.783094   0.055151  0.659466  0.100948  0.141016  0.785284  0.340534

fold     AUROC  accuracy  precision    recall        F1     AUPRC       TPR       FPR
data subset                                                                                          
set  8       3.02765  0.135461  0.068243   0.044508  0.462961  0.077117  0.123463  0.073729  0.269921
     45      3.02765  0.087543  0.059741   0.023401  0.182321  0.040909  0.055891  0.061222  0.182321

In [14]:
# CV for set transformer - random embedding results - on whole dataset and those with rare co-morbidities

# Get CV folds and create dictionarys 
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
fold_n = 0
train_index_dict = {}
test_index_dict = {}
test_index_dict_8 = {}
test_index_dict_45 = {}

for train_index, test_index in cv.split(feature_array, mortality_array):
    fold_n += 1
    train_index_dict[fold_n] = train_index
    test_index_dict[fold_n] = test_index
    # Filter for rare diseases in test set
    test_index_8 = np.array(list(set(test_index.tolist()) & set(index_list_8)))
    test_index_45 = np.array(list(set(test_index.tolist()) & set(index_list_45)))
    test_index_dict_8[fold_n] = test_index_8
    test_index_dict_45[fold_n] = test_index_45

In [15]:
# del for memmory 
del test_index
del train_index
del patient_df
del index
del fold_n
del code
del code_list
del path
del n 
del n2
del max_len
gc.collect()

0

In [16]:
# Run
best_test_auroc = 0
results_df = pd.DataFrame()
for fold in range(10):
    fold += 1
    if fold < 10:
        # Get train, validation and test sets
        train_embeddings = feature_array[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_embeddings = feature_array[test_index_dict[fold+1]]
        test_embeddings = feature_array[test_index_dict[fold]]
        test_embeddings_8 = feature_array[test_index_dict_8[fold]]
        test_embeddings_45 = feature_array[test_index_dict_45[fold]]

        train_labels = mortality_array[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_labels = mortality_array[test_index_dict[fold+1]]
        test_labels = mortality_array[test_index_dict[fold]]
        test_labels_8 = mortality_array[test_index_dict_8[fold]]
        test_labels_45 = mortality_array[test_index_dict_45[fold]]    

        # Split masks
        train_padding_mask = mask[np.setdiff1d(train_index_dict[fold], test_index_dict[fold+1])]
        val_padding_mask = mask[test_index_dict[fold+1]]
        test_padding_mask = mask[test_index_dict[fold]]
        test_padding_mask_8 = mask[test_index_dict_8[fold]]
        test_padding_mask_45 = mask[test_index_dict_45[fold]]

    else: # Change here to fist cv split for final fold
        # Get train, validation and test sets
        train_embeddings = feature_array[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_embeddings = feature_array[test_index_dict[1]]
        test_embeddings = feature_array[test_index_dict[fold]]
        test_embeddings_8 = feature_array[test_index_dict_8[fold]]
        test_embeddings_45 = feature_array[test_index_dict_45[fold]]

        train_labels = mortality_array[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_labels = mortality_array[test_index_dict[1]]
        test_labels = mortality_array[test_index_dict[fold]]
        test_labels_8 = mortality_array[test_index_dict_8[fold]]
        test_labels_45 = mortality_array[test_index_dict_45[fold]]

        # Split masks
        train_padding_mask = mask[np.setdiff1d(train_index_dict[fold], test_index_dict[1])]
        val_padding_mask = mask[test_index_dict[1]]
        test_padding_mask = mask[test_index_dict[fold]]
        test_padding_mask_8 = mask[test_index_dict_8[fold]]
        test_padding_mask_45 = mask[test_index_dict_45[fold]]
    
    # Create datasets
    train_dataset = DiseaseDataset(train_embeddings.astype(np.float32), train_labels.astype(np.float32), train_padding_mask)
    val_dataset = DiseaseDataset(val_embeddings.astype(np.float32), val_labels.astype(np.float32), val_padding_mask)
    test_dataset = DiseaseDataset(test_embeddings.astype(np.float32), test_labels.astype(np.float32), test_padding_mask)
    test_dataset_8 = DiseaseDataset(test_embeddings_8.astype(np.float32), test_labels_8.astype(np.float32), test_padding_mask_8)
    test_dataset_45 = DiseaseDataset(test_embeddings_45.astype(np.float32), test_labels_45.astype(np.float32), test_padding_mask_45)

    # Define batch size
    batch_size = 512

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    test_loader_8 = DataLoader(test_dataset_8, batch_size=batch_size)
    test_loader_45 = DataLoader(test_dataset_45, batch_size=batch_size)

    # Create weight for loss
    unique, counts = np.unique(train_labels, return_counts=True)
    pos_weight = torch.Tensor([(counts[0] / counts[1])])

    # Define model
    model = SetTransformer(dim_input=128, num_outputs=1, dim_output=1, num_inds=32, dim_hidden=160, num_heads=4, ln=False)

    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Run
    best_valid_loss = float('inf')
    best_valid_auroc = 0
    num_epochs = 10

    for epoch in range(num_epochs):

        start_time = time.time()

        train_loss, train_auroc, train_predictions, train_labels_out, train_pma_final = train(model, train_loader, optimizer, criterion)
        valid_loss, valid_auroc, valid_predictions, valid_labels_out, valid_pma_final = evaluate(model, val_loader, criterion)
        
        if epoch % 1 == 0:
            print('Epoch:', epoch)
            print(f'train loss: {train_loss:.3f}')
            print(f'train AUROC: {train_auroc:.3f}')
            print(f'valid loss: {valid_loss:.3f}')
            print(f'valid AUROC: {valid_auroc:.3f}')
        
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss  
            print('BEST VALID LOSS:', best_valid_loss)

        if valid_auroc > best_valid_auroc:
            best_valid_auroc = valid_auroc
            print('BEST VALID AUROC:', best_valid_auroc)
            print('UPDATED BEST INTERMEDIATE MODEL')
            torch.save(model.state_dict(), f'intermediate_set_transformer_mortality.pt')

    # -----------------------------
    # Evaluate best model on test set
    # -----------------------------

    model.load_state_dict(torch.load(f'intermediate_set_transformer_mortality.pt'))

    test_loss, test_auroc, test_predictions, test_labels_out, test_pma_final = evaluate(model, test_loader, criterion)
    test_loss_8, test_auroc_8, test_predictions_8, test_labels_out_8, test_pma_final_8 = evaluate(model, test_loader_8, criterion)
    test_loss_45, test_auroc_45, test_predictions_45, test_labels_out_45, test_pma_final_45 = evaluate(model, test_loader_45, criterion)

    # del for memmory 
    del train_embeddings
    del val_embeddings
    del train_labels
    del val_labels
    del train_padding_mask
    del val_padding_mask
    del train_loader
    del val_loader
    del train_predictions
    del train_labels_out
    del train_pma_final
    del valid_predictions
    del valid_labels_out
    del valid_pma_final
    gc.collect()

    print(f'test predictions: {test_predictions.mean().item():.3f}') 
    print(f'test loss: {test_loss:.3f}')
    print(f'test AUROC: {test_auroc:.3f}')

    print(f'test predictions 8: {test_predictions_8.mean().item():.3f}') 
    print(f'test loss 8: {test_loss_8:.3f}')
    print(f'test AUROC 8: {test_auroc_8:.3f}')

    print(f'test predictions 45: {test_predictions_45.mean().item():.3f}') 
    print(f'test loss 45: {test_loss_45:.3f}')
    print(f'test AUROC 45: {test_auroc_45:.3f}')

    if test_auroc > best_test_auroc:
        best_test_auroc = test_auroc
        print('BEST TEST AUROC:', best_test_auroc)
        print('UPDATED BEST MODEL')
        torch.save(model.state_dict(), f'set_transformer_random_mortality.pt')

    # Get results
    # AUC
    aucroc = roc_auc_score(test_labels_out, test_predictions)
    # Accuracy
    accuracy = accuracy_score(test_labels_out, test_predictions.round())
    # Precision
    precision = precision_score(test_labels_out, test_predictions.round())
    # Recall
    recall = recall_score(test_labels_out, test_predictions.round())
    # AUPRC
    auprc = average_precision_score(test_labels_out, test_predictions)
    # F1
    f1 = f1_score(test_labels_out, test_predictions.round())
    # TPR and FPR
    cm = confusion_matrix(test_labels_out, test_predictions.round())
    _tp = cm[0, 0]
    _fn = cm[0, 1]
    _fp = cm[1, 0]
    _tn = cm[1, 1]
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'set', 'fold': fold, 'subset':'overall', 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)

    # Get results - rare
    # AUC
    try:
        aucroc = roc_auc_score(test_labels_out_8, test_predictions_8)
    except:
        aucroc = np.nan
    # Accuracy
    accuracy = accuracy_score(test_labels_out_8, test_predictions_8.round())
    # Precision
    precision = precision_score(test_labels_out_8, test_predictions_8.round())
    # Recall
    recall = recall_score(test_labels_out_8, test_predictions_8.round())
    # AUPRC
    try:
        auprc = average_precision_score(test_labels_out_8, test_predictions_8)
    except:
        auprc = np.nan
    # F1
    f1 = f1_score(test_labels_out_8, test_predictions_8.round())
    # TPR and FPR
    cm = confusion_matrix(test_labels_out_8, test_predictions_8.round())
    try:
        _tp = cm[0, 0]
    except:
        _tp = np.nan
    try:
        _fn = cm[0, 1]
    except:
        _fn = np.nan
    try:
        _fp = cm[1, 0]
    except:
        _fp = np.nan
    try:
        _tn = cm[1, 1]
    except:
        _tn = np.nan
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'set', 'fold': fold, 'subset': 8, 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)

    # Get results
    # AUC
    try:
        aucroc = roc_auc_score(test_labels_out_45, test_predictions_45)
    except:
        aucroc = np.nan
    # Accuracy
    accuracy = accuracy_score(test_labels_out_45, test_predictions_45.round())
    # Precision
    precision = precision_score(test_labels_out_45, test_predictions_45.round())
    # Recall
    recall = recall_score(test_labels_out_45, test_predictions_45.round())
    # AUPRC
    try:
        auprc = average_precision_score(test_labels_out_45, test_predictions_45)
    except:
        auprc = np.nan
    # F1
    f1 = f1_score(test_labels_out_45, test_predictions_45.round())
    # TPR and FPR
    cm = confusion_matrix(test_labels_out_45, test_predictions_45.round())
    try:
        _tp = cm[0, 0]
    except:
        _tp = np.nan
    try:
        _fn = cm[0, 1]
    except:
        _fn = np.nan
    try:
        _fp = cm[1, 0]
    except:
        _fp = np.nan
    try:
        _tn = cm[1, 1]
    except:
        _tn = np.nan
    tpr = _tp / (_tp + _fn)
    fpr = _fp / (_tn + _fp)

    new_row = {'data': 'set', 'fold': fold, 'subset': 45, 'AUROC': aucroc, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'F1': f1, 'AUPRC': auprc, 'TPR': tpr, 'FPR': fpr}
    results_df = results_df.append(new_row, ignore_index=True)


100%|██████████| 18/18 [00:04<00:00,  3.74it/s]


Epoch: 0
train loss: 1.177
train AUROC: 0.737
valid loss: 1.320
valid AUROC: 0.759
BEST VALID LOSS: 1.320160620742374
BEST VALID AUROC: 0.7593043014858653
UPDATED BEST INTERMEDIATE MODEL
Epoch: 1
train loss: 1.186
train AUROC: 0.748
valid loss: 1.120
valid AUROC: 0.778
BEST VALID LOSS: 1.1204609904024336
BEST VALID AUROC: 0.7779186000842953
UPDATED BEST INTERMEDIATE MODEL
Epoch: 2
train loss: 1.061
train AUROC: 0.797
valid loss: 1.071
valid AUROC: 0.800
BEST VALID LOSS: 1.0706020891666412
BEST VALID AUROC: 0.8004143444606068
UPDATED BEST INTERMEDIATE MODEL
Epoch: 3
train loss: 1.029
train AUROC: 0.814
valid loss: 1.052
valid AUROC: 0.814
BEST VALID LOSS: 1.0522677169905768
BEST VALID AUROC: 0.8143879454784896
UPDATED BEST INTERMEDIATE MODEL
Epoch: 4
train loss: 1.040
train AUROC: 0.808
valid loss: 1.064
valid AUROC: 0.814
Epoch: 5
train loss: 0.999
train AUROC: 0.827
valid loss: 1.059
valid AUROC: 0.821
BEST VALID AUROC: 0.8207281685390421
UPDATED BEST INTERMEDIATE MODEL
Epoch: 6
train

<All keys matched successfully>

100%|██████████| 2/2 [00:00<00:00,  5.92it/s]


0

...

<All keys matched successfully>

100%|██████████| 1/1 [00:00<00:00,  3.65it/s]


0

test predictions: 0.304
test loss: 1.024
test AUROC: 0.824
test predictions 8: 0.311
test loss 8: 0.513
test AUROC 8: nan
test predictions 45: 0.301
test loss 45: 1.235
test AUROC 45: 0.669


/tmp/ipykernel_5678/932404822.py:188: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/tmp/ipykernel_5678/932404822.py:228: RuntimeWarning: invalid value encountered in long_scalars
  fpr = _fp / (_tn + _fp)
/tmp/ipykernel_5678/932404822.py:231: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

In [17]:
results_df

,data,fold,subset,AUROC,accuracy,precision,recall,F1,AUPRC,TPR,FPR
0,set,1,overall,0.801023,0.934336,0.122153,0.257642,0.165730,0.100178,0.951911,0.742358
1,set,1,8,NaN,0.892473,0.000000,0.000000,0.000000,-0.000000,0.892473,NaN
2,set,1,45,0.615291,0.934664,0.000000,0.000000,0.000000,0.018356,0.944954,1.000000
3,set,2,overall,0.809279,0.793832,0.083078,0.711790,0.148791,0.115004,0.795962,0.288210
4,set,2,8,NaN,0.746667,0.000000,0.000000,0.000000,-0.000000,0.746667,NaN
5,set,2,45,0.838454,0.766114,0.031008,0.666667,0.059259,0.041074,0.767225,0.333333
6,set,3,overall,0.852223,0.839155,0.101430,0.681223,0.176570,0.127085,0.843257,0.318777
7,set,3,8,0.989130,0.817204,0.055556,1.000000,0.105263,0.500000,0.815217,0.000000
8,set,3,45,0.726943,0.904854,0.045455,0.222222,0.075472,0.093181,0.916996,0.777778
9,set,4,overall,0.828857,0.906810,0.135553,0.495652,0.212885,0.141549,0.917536,0.504348


In [18]:
results_df.groupby(['data', 'subset']).mean()
results_df.groupby(['data', 'subset']).std()

fold     AUROC  accuracy  precision    recall        F1     AUPRC       TPR       FPR
data subset                                                                                        
set  8         5.5  0.556078  0.760136   0.041033  0.283333  0.058679  0.159266  0.763060  0.595238
     45        5.5  0.634672  0.786020   0.038258  0.333016  0.060523  0.051705  0.793498  0.666984
     overall   5.5  0.804258  0.772035   0.084724  0.613321  0.141723  0.108583  0.776162  0.386679

fold     AUROC  accuracy  precision    recall        F1     AUPRC       TPR       FPR
data subset                                                                                           
set  8        3.02765  0.327496  0.205080   0.078267  0.416111  0.094579  0.222920  0.223687  0.449868
     45       3.02765  0.107444  0.187253   0.040618  0.281479  0.052552  0.025499  0.195832  0.281479
     overall  3.02765  0.031446  0.165882   0.030799  0.204433  0.041329  0.025596  0.174637  0.204433